In [2]:
import os
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.optim as optim
import torch.nn as nn
from PIL import Image
import os

In [3]:
image_folder = "WFLW_images"
selected_indices = list(range(60, 64)) + list(range(68, 72)) + [96, 97] # aici selectam indicii pt perechile de (x,y) de keypoints pt ochi
selected_indices_x = [i*2 for i in selected_indices]
selected_indices_y = [i*2+1 for i in selected_indices]
selected_indices = selected_indices_x + selected_indices_y
selected_indices.sort()
print(f"Selected keypoint indices: {selected_indices}") # print la toate perechile

train_keypoints_file = "list_98pt_rect_attr_train.txt"
test_keypoints_file = "list_98pt_rect_attr_test.txt"

columns = [
    f"x{i//2}" if i % 2 == 0 else f"y{i//2}" for i in range(196)
] + ["x_min_rect", "y_min_rect", "x_max_rect", "y_max_rect",
     "pose", "expression", "illumination", "make_up", "occlusion", "blur", "image_name"]

keypoints_train_df = pd.read_csv(train_keypoints_file, sep=r'\s+', names=columns)
keypoints_test_df = pd.read_csv(test_keypoints_file, sep=r'\s+', names=columns)

train_keypoints_df = keypoints_train_df.iloc[:, selected_indices].copy()  # adaugam mizeriile de x,y keypoints
train_keypoints_df['image_name'] = keypoints_train_df['image_name'].copy() # adaugam numele imaginii ca sa o putem cauta in folder

test_keypoints_df = keypoints_test_df.iloc[:, selected_indices].copy() 
test_keypoints_df['image_name'] = keypoints_test_df['image_name'].copy()


train_keypoints_df = train_keypoints_df[['image_name'] + [col for col in train_keypoints_df.columns if col != 'image_name']] # punem numele pe prima coloana sa fie standard
test_keypoints_df = test_keypoints_df[['image_name'] + [col for col in test_keypoints_df.columns if col != 'image_name']]

train_keypoints_df.to_csv('data/filtered_train_keypoints.csv', index=False) # facem csv-uri noi pe care le vom folosi pt a crea tensorii
test_keypoints_df.to_csv('data/filtered_test_keypoints.csv', index=False)

Selected keypoint indices: [120, 121, 122, 123, 124, 125, 126, 127, 136, 137, 138, 139, 140, 141, 142, 143, 192, 193, 194, 195]


In [4]:
class FacialKeypointsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string) - dataframe-ul
            root_dir (string) - folder cu imagini
            transform (callable, optional) - vom folosi transform pt. normalizare etc (care dintr-un motiv sau altul cred ca nu merge)
        """
        self.keypoints_frame = pd.read_csv(csv_file)  
        self.root_dir = root_dir 
        self.transform = transform  

    def __len__(self):
        return len(self.keypoints_frame) 

    def __getitem__(self, idx):
        img_name = self.keypoints_frame.iloc[idx, 0]  # deci fiecare element o sa aiba nume
        img_path = os.path.join(self.root_dir, img_name)  
        image = Image.open(img_path)  

        keypoints = self.keypoints_frame.iloc[idx, 1:].values  # & coordonate pt ochi
        keypoints = keypoints.astype('float').reshape(-1, 2) 
  
        sample = {'image': image, 'keypoints': keypoints}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [5]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # input default pt. resnet
    transforms.ToTensor(),  # facem tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # astea cica s-ar folosi ca sa normalizam imagini rgb in torchvision, mi se pare o mare mierda
])

train_kp = "data\\filtered_train_keypoints.csv"
test_kp = "data\\filtered_test_keypoints.csv" 

train_dataset = FacialKeypointsDataset(csv_file=train_kp,
                                        root_dir=image_folder,
                                        transform=data_transform)

test_dataset = FacialKeypointsDataset(csv_file=test_kp,
                                       root_dir=image_folder,
                                       transform=data_transform)

In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # astea-s generatoarele de pe tensorflow ish
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Verificăm câteva statistici pentru dataset-uri
print(f'Numărul de imagini de antrenament: {len(train_dataset)}')
print(f'Numărul de imagini de test: {len(test_dataset)}')

for i, sample in enumerate(train_loader):
    if i == 0:
        print(f"Imagini (batch): {sample['image'].size()}")
        print(f"Keypoints (batch): {sample['keypoints'].size()}")

Numărul de imagini de antrenament: 7500
Numărul de imagini de test: 2500
Imagini (batch): torch.Size([32, 3, 224, 224])
Keypoints (batch): torch.Size([32, 10, 2])


In [7]:
resnet18 = models.resnet18(pretrained=True) 

num_keypoints = 10 
resnet18.fc = nn.Linear(resnet18.fc.in_features, num_keypoints * 2) # deci resnet-u ar trebui sa primeasca batch de imagini la input si sa scoata 20 de keypoints adica 20 neuroni la iesire

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # aici premium pentru aia bogati cu gpu 
resnet18.to(device)

criterion = nn.MSELoss() 
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)

C:\Users\Rares\anaconda3\envs\torchgpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Rares\anaconda3\envs\torchgpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
!pip install tqdm # i-am dat la chat sa-mi faca cu tqdm pt ca nu puteam sa stau sa ma uit la mizeria aia progres
from tqdm import tqdm

# Funcția de training actualizată cu bara de progres
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        with tqdm(train_loader, unit="batch", desc=f"Epoch {epoch+1}/{num_epochs}") as tepoch:
            for sample in tepoch:
                images = sample['image'].to(device).float()  # aici se convertesc toate datele in float pt. ca asa se folosesc la resnet (imi dadea eroare)
                keypoints = sample['keypoints'].to(device).float() 
                

                optimizer.zero_grad()
                outputs = model(images)
                
                loss = criterion(outputs, keypoints.view(-1, num_keypoints * 2)) # eu cred ca loss-ul e mare si din cauza acestor kp ca practic pot sa aiba si valori gen 200 daca asta clasifica ca pula
                
                loss.backward()
                
                optimizer.step()
                
                running_loss += loss.item()
                
                tepoch.set_postfix(loss=running_loss / (tepoch.n + 1))
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')


train_model(resnet18, train_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10: 100%|███████████████████████████████████████████████████| 235/235 [01:56<00:00,  2.02batch/s, loss=1.77e+5]


Epoch 1/10, Loss: 177060.82697805853


Epoch 2/10: 100%|███████████████████████████████████████████████████| 235/235 [01:57<00:00,  2.00batch/s, loss=7.82e+4]


Epoch 2/10, Loss: 78186.14509640957


Epoch 3/10: 100%|███████████████████████████████████████████████████| 235/235 [01:59<00:00,  1.97batch/s, loss=6.68e+4]


...

Epoch 7/10, Loss: 55818.59919381649


Epoch 8/10: 100%|████████████████████████████████████████████████████| 235/235 [01:56<00:00,  2.02batch/s, loss=5.3e+4]


Epoch 8/10, Loss: 52989.75903424202


Epoch 9/10:  26%|█████████████▍                                      | 61/235 [00:30<01:26,  2.01batch/s, loss=5.43e+4]


KeyboardInterrupt: 

In [ ]:
# Nora pls help i am tired i am on fucking pills